In [36]:
import requests
import json

def fetch_all_qualifications():
    url = "https://kgr95pqfu5-2.algolianet.com/1/indexes/*/queries"

    headers = {
        "Accept": "*/*",
        "Content-Type": "application/x-www-form-urlencoded",
        "Origin": "https://ocnlondon.org.uk",
        "Referer": "https://ocnlondon.org.uk/",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
        "x-algolia-api-key": "ea88db3cb6574278728fe21a9db87f98",
        "x-algolia-application-id": "KGR95PQFU5"
    }

    all_courses = {}
    page_number = 0

    while True:
        payload = {
            "requests": [
                {
                    "indexName": "wp_qualification",
                    "params": f"facets=[\"category\",\"course_length\",\"funding\",\"has_learning_resources\",\"level\",\"scheme\"]"
                              f"&highlightPostTag=__/ais-highlight__"
                              f"&highlightPreTag=__ais-highlight__"
                              f"&maxValuesPerFacet=10"
                              f"&page={page_number}"
                              f"&query="
                              f"&tagFilters="
                }
            ]
        }

        response = requests.post(url, headers=headers, data=json.dumps(payload))

        if response.status_code == 200:
            data = response.json()
            results = data.get("results", [])[0].get("hits", [])

            if not results:
                break  # Stop when no more results

            for result in results:
                course_code = result.get("course_code", f"Unknown_{page_number}")  # Use course_code or fallback
                qualification_url = result.get("url", "No URL")
                all_courses[course_code] = qualification_url

            page_number += 1  # Move to the next page

        else:
            print(f"Failed to fetch data. Status Code: {response.status_code}")
            break
    print("len(all_courses)", len(all_courses))
    return json.dumps(all_courses, indent=4)

# Run the function and print results
courses_json = fetch_all_qualifications()
print(courses_json)


len(all_courses) 371
{
    "610/1126/5": "https://ocnlondon.org.uk/qualifications/esol/entry-level-certificate-in-esol-international-entry-1-cefr-a1/",
    "603/5864/6": "https://ocnlondon.org.uk/qualifications/empl-skills/e1-cert-entry-employment/",
    "603/5863/4": "https://ocnlondon.org.uk/qualifications/empl-skills/e1-award-entry-employment/",
    "603/5419/7": "https://ocnlondon.org.uk/qualifications/supported-learning/e1-dip-skills-independence-empl/",
    "603/5416/1": "https://ocnlondon.org.uk/qualifications/supported-learning/e1-ext-cert-skills-independence-empl/",
    "603/5413/6": "https://ocnlondon.org.uk/qualifications/supported-learning/e1-cert-skills-independence-empl/",
    "603/5410/0": "https://ocnlondon.org.uk/qualifications/supported-learning/e1-award-skills-independence-empl/",
    "603/4707/7": "https://ocnlondon.org.uk/qualifications/esol/e1-award-sfl-speaking-listening/",
    "603/4712/0": "https://ocnlondon.org.uk/qualifications/esol/e1-award-sfl-writing/",
  

In [39]:
import requests
from bs4 import BeautifulSoup
import json
import os



courses=json.loads(courses_json)

# Directory to save PDFs
download_dir = r"C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs"
os.makedirs(download_dir, exist_ok=True)

# CSS selector for the PDF link
pdf_selector = "body > main > section:nth-child(3) > div > div > div.flex-1.mb-16.lg\\:mb-0.lg\\:mr-16.overflow-auto > div.mb-16.space-y-6 > a"

for course_code, course_url in courses.items():
    try:
        # Fetch the course page
        response = requests.get(course_url)
        soup = BeautifulSoup(response.text, "html.parser")

        # Find the PDF link
        pdf_element = soup.select_one(pdf_selector)
        if not pdf_element:
            print(f"No PDF found for {course_code}")
            continue

        pdf_url = pdf_element.get("href")
        if not pdf_url.startswith("http"):
            pdf_url = f"https://ocnlondon.org.uk{pdf_url}"  # Convert relative URL to absolute

        # Download the PDF
        pdf_response = requests.get(pdf_url)
        pdf_path = os.path.join(download_dir, f"{course_code.replace('/','-')}.pdf")

        # Save the PDF
        with open(pdf_path, "wb") as f:
            f.write(pdf_response.content)

        print(f"Downloaded: {pdf_path}")

    except Exception as e:
        print(f"Error processing {course_code}: {e}")


Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\610-1126-5.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\603-5864-6.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\603-5863-4.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\603-5419-7.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\603-5416-1.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\603-5413-6.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\603-5410-0.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\603-4707-7.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\603-4712-0.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\603-4622-X.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\603-4717-X.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\601-7111-X.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\601-6635-6.pdf
Downloaded: C:\Users\amith\Kenpath\OFQUAL\ocnlondon_pdfs\601-602